### 📌 1. Jira의 이슈 불러오기
> 🔗 1. API 호출을 통해 Jira 데이터 가져오기
- Jira의 이슈들을 API와 JQL을 통해 호출

In [3]:
#!/usr/bin/env python3
"""
Jira에서 이슈들을 가져와서 JSONL 형태로 변환하는 스크립트 (임베딩용)
"""

from typing import Dict, List, Optional, Any
import requests
import json
import time
import os
from datetime import datetime
import torch
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Any
import numpy as np

# --- 설정 ---
JIRA_URL = "https://jira.suprema.co.kr"
JIRA_USERNAME =
JIRA_PASSWORD = 
JIRA_AUTH = (JIRA_USERNAME, JIRA_PASSWORD)
# 최종 결과가 저장될 폴더 이름
OUTPUT_FOLDER = "jira_issues_output" # 저장 폴더 이름을 더 명확하게 변경

# 검색할 JQL 쿼리
JQL_QUERY = 'project = "COMMONR" AND issuetype = "Test"'

SyntaxError: invalid syntax (600325865.py, line 19)

In [ ]:
%pip install -U sentence-transformers

In [ ]:
def get_all_jira_issue_keys(jql):
    """JQL로 모든 이슈를 검색하여 이슈 키 리스트를 반환합니다."""
    all_issue_keys = []
    start_at = 0
    max_results = 100

    print(f"JQL로 이슈 검색을 시작합니다: {jql}")

    while True:
        url = f"{JIRA_URL}/rest/api/2/search"
        headers = {"Accept": "application/json"}
        params = {
            'jql': jql,
            'fields': 'key',
            'startAt': start_at,
            'maxResults': max_results
        }

        try:
            response = requests.get(url, headers=headers, params=params, auth=JIRA_AUTH)
            response.raise_for_status()
            data = response.json()
            issues_on_page = data.get('issues', [])
            
            if not issues_on_page:
                break
            
            keys_on_page = [issue['key'] for issue in issues_on_page]
            all_issue_keys.extend(keys_on_page)
            
            start_at += len(issues_on_page)
            print(f"  -> 현재까지 {len(all_issue_keys)} / {data['total']} 개 이슈 키를 가져왔습니다...")
            
        except requests.exceptions.RequestException as e:
            print(f"Jira 이슈 검색 오류: {e}")
            return None
            
    return all_issue_keys

### 📌 2. JSON 파일 열 정제
> 🔗 2. JSON 파일을 임베딩 모델에 임베딩하기 좋은 형식으로 저장
- Jira의 이슈를 가져와서 임베딩 및 langchain에 가공하기 좋은 형식으로 저장

In [ ]:
def convert_to_embedding_format(issue_key: str) -> Optional[List[Dict[str, Any]]]:
    """
    하나의 이슈 키에 대한 모든 정보를 Jira API를 통해 가져와서
    임베딩 및 ChromaDB 저장에 최적화된 형식으로 변환ㅁ합니다.
    
    Args:
        issue_key (str): Jira 이슈 키
        
    Returns:
        Optional[List[Dict[str, Any]]]: 임베딩용 객체들의 리스트
        각 객체는 {'id', 'document', 'metadata'} 구조
    """
    url = f"{JIRA_URL}/rest/api/2/issue/{issue_key}"
    headers = {"Accept": "application/json"}
    
    try:
        response = requests.get(url, headers=headers, auth=JIRA_AUTH)
        response.raise_for_status()
        issue_data = response.json()
        
        if not issue_data:
            return None
        
        # 이슈 데이터 추출
        fields = issue_data.get('fields', {})
        description = fields.get('description', '')
        custom_field_10004 = fields.get('customfield_10004', {})
        steps_list = custom_field_10004.get('steps', [])
        
        embedding_objects = []
        
        # 각 테스트 스텝별로 임베딩 객체 생성
        if isinstance(steps_list, list):
            for item in steps_list:
                if isinstance(item, dict):
                    index = item.get('index', '')
                    step = item.get('step', '')
                    data_item = item.get('data', '')
                    expected_result = item.get('result', '')
                    
                    # page_content: test, step, data, expected result만
                    content_parts = []
                    if step:
                        content_parts.append(f"Test Step: {step}")
                    if data_item:
                        content_parts.append(f"Test Data: {data_item}")
                    if expected_result:
                        content_parts.append(f"Expected Result: {expected_result}")
                    
                    if content_parts:
                        step_object = {
                            "id": f"{issue_key}_step_{index}",
                            "document": "\n".join(content_parts),  # ChromaDB 용어 사용
                            "metadata": {
                                "issue_key": issue_key,
                                "step_index": str(index),  # ChromaDB는 문자열 선호
                                "source": "jira_test_step"  # 데이터 소스 식별용
                            }
                        }
                        embedding_objects.append(step_object)
        
        return embedding_objects
        
    except requests.exceptions.RequestException as e:
        print(f"'{issue_key}'의 정보 조회 중 오류 발생: {e}")
        return None

In [ ]:
def json_to_save_file(json_data,issue_key) : 
    #1. 결과를 저장할 폴더 생성
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f" {issue_key} 이슈의 정보를 json으로 저장을 시작합니다.")
    print(f"저장 폴더: '{os.path.abspath(OUTPUT_FOLDER)}'")

    #2. json 파일로 저장장
    try:
        output_filepath = os.path.join(OUTPUT_FOLDER, f"{issue_key}.json")
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)
        return True
    except IOError as e:
        print(f"'{output_filepath}' 파일 저장 중 오류 발생: {e}")
        return False

### 📌 3.JSON 파일 임베딩 모델을 통해 임베딩
> 🔗 JSON 파일을 임베딩 모델을 통해서 임베딩
- e5-multilingual 임베딩 모델을 통해서 임베딩

In [1]:
import os
import json
import torch
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Any


def embed_jira_documents_with_e5(jira_folder_path: str = "jira_issues_output",
                                model_name: str = "intfloat/multilingual-e5-large") -> List[Dict[str, Any]]:
    """
    jira_issues_output 폴더의 JSON 파일들을 읽어서 E5 모델로 임베딩합니다.
    
    Args:
        jira_folder_path (str): JIRA JSON 파일들이 있는 폴더 경로
        model_name (str): 사용할 E5 모델명
    
    Returns:
        List[Dict[str, Any]]: LangChain용으로 최적화된 임베딩 객체들
            각 객체는 {'id', 'page_content', 'metadata', 'embedding'} 구조
    """
    # 1. JSON 파일 읽기
    if not os.path.exists(jira_folder_path):
        raise FileNotFoundError(f"폴더를 찾을 수 없습니다: {jira_folder_path}")
    
    json_files = [f for f in os.listdir(jira_folder_path) if f.endswith('.json')]
    if not json_files:
        raise ValueError(f"폴더에 JSON 파일이 없습니다: {jira_folder_path}")
    
    all_documents = []
    for json_file in json_files:
        file_path = os.path.join(jira_folder_path, json_file)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, list):
                all_documents.extend(data)
            else:
                all_documents.append(data)
    
    # 2. LangChain 형식으로 변환 (예외처리 포함)
    embedding_objects = []
    for doc in all_documents:
        # 필수 키 체크 및 예외처리
        if not doc or 'id' not in doc or 'document' not in doc or 'metadata' not in doc:
            continue  # 빈 객체나 필수 키가 없으면 건너뛰기
            
        if not doc['document'].strip():  # 빈 문서도 건너뛰기
            continue
            
        embedding_obj = {
            'id': doc['id'],
            'page_content': doc['document'],
            'metadata': doc['metadata']
        }
        embedding_objects.append(embedding_obj)
    
    # 3. E5 모델 로드 및 임베딩
    print(f"E5 모델 로딩 중: {model_name}")
    model = SentenceTransformer(model_name)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    print(f"사용 디바이스: {device}")
    
    # page_content 추출하여 임베딩
    documents = [obj['page_content'] for obj in embedding_objects]
    
    print(f"총 {len(documents)}개 문서 임베딩 시작...")
    
    prefixed_documents = [f"passage: {doc}" for doc in documents]
    
    embeddings = model.encode(
        prefixed_documents,
        batch_size=8,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    
    print(f"임베딩 완료! 벡터 차원: {embeddings.shape[1]}")
    
    # 4. 임베딩 결과 추가
    result_objects = []
    for i, obj in enumerate(embedding_objects):
        embedded_obj = obj.copy()
        embedded_obj['embedding'] = embeddings[i].tolist()
        result_objects.append(embedded_obj)
    
    return result_objects

### 📌 4. 벡터 DB를 통해서 저장
> 🔗 해당 임베딩한 것을 벡터 DB를 통해서 저장
- Chroma DB를 사용해서 임베딩 값 저장



In [2]:
import chromadb
from typing import List, Dict, Any


def save_embeddings_to_chroma(embedded_objects: List[Dict[str, Any]], 
    collection_name: str = "jira_test_cases",
    persist_directory: str = "./chroma_db") -> chromadb.Collection:
    """
    임베딩된 객체들을 ChromaDB에 저장합니다.
    
    Args:
        embedded_objects (List[Dict[str, Any]]): 임베딩이 포함된 객체들
            각 객체는 {'id', 'page_content', 'metadata', 'embedding'} 구조
        collection_name (str): ChromaDB 컬렉션 이름
        persist_directory (str): ChromaDB 데이터 저장 경로
    
    Returns:
        chromadb.Collection: 생성된 ChromaDB 컬렉션
    """
    # ChromaDB 클라이언트 생성 (영구 저장)
    client = chromadb.PersistentClient(path=persist_directory)
    
    # 기존 컬렉션이 있으면 삭제 후 새로 생성
    try:
        client.delete_collection(name=collection_name)
        print(f"기존 컬렉션 '{collection_name}' 삭제됨")
    except:
        pass
    
    # 새 컬렉션 생성
    collection = client.create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}  # 코사인 유사도 사용
    )
    
    # 데이터 분리
    ids = [obj['id'] for obj in embedded_objects]
    documents = [obj['page_content'] for obj in embedded_objects]
    metadatas = [obj['metadata'] for obj in embedded_objects]
    embeddings = [obj['embedding'] for obj in embedded_objects]
    
    # ChromaDB에 배치 추가
    collection.add(
        ids=ids,
        documents=documents,
        metadatas=metadatas,
        embeddings=embeddings
    )
    
    print(f"✅ {len(embedded_objects)}개 문서가 ChromaDB 컬렉션 '{collection_name}'에 저장되었습니다.")
    print(f"📁 저장 위치: {persist_directory}")
    
    return collection

embedded_data = embed_jira_documents_with_e5()
collection = save_embeddings_to_chroma(embedded_data)

E5 모델 로딩 중: intfloat/multilingual-e5-large
사용 디바이스: cpu
총 2632개 문서 임베딩 시작...


Batches:   0%|          | 0/329 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


임베딩 완료! 벡터 차원: 1024
✅ 2632개 문서가 ChromaDB 컬렉션 'jira_test_cases'에 저장되었습니다.
📁 저장 위치: ./chroma_db


### 📌 5-1. langchain을 이용한 구축 with LM Studio
> 🔗 langchain을 이용해서 저장한 임베딩 값을 Langchain, LM Studio와 연계하여 최대의 값 도출
- langchain을 이용하여 chroma DB 결과값 QA RAG로 구현
- 해당 RAG LM Studio LLM과 연동하여 기대값에 대한 결과값 도출

In [ ]:
import chromadb
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from typing import List, Dict, Any, Optional
import json
import requests
import warnings
import datetime

# FutureWarning 무시
warnings.filterwarnings("ignore", category=FutureWarning)


class LMStudioLLM(LLM):
    """LM Studio와 연동하는 LangChain 호환 LLM 클래스"""
    
    base_url: str = "http://127.0.0.1:1234/v1"
    model_name: str = "qwen/qwen3-8b"
    temperature: float = 0.1
    max_tokens: int = 2048
    
    def __init__(self, 
                 base_url: str = "http://127.0.0.1:1234/v1",
                 model_name: str = "qwen/qwen3-8b",
                 temperature: float = 0.1,
                 max_tokens: int = 2048,
                 **kwargs):
        super().__init__(**kwargs)
        self.base_url = base_url.rstrip('/')
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self._test_connection()
    
    def _test_connection(self):
        """LM Studio 연결 테스트"""
        try:
            response = requests.get(f"{self.base_url}/models", timeout=5)
            if response.status_code == 200:
                models = response.json()
                print(f"✅ LM Studio 연결 성공! 사용 가능한 모델: {len(models.get('data', []))}개")
            else:
                print(f"⚠️ LM Studio 연결 상태 확인 필요: {response.status_code}")
        except Exception as e:
            print(f"❌ LM Studio 연결 실패: {e}")
    
    @property
    def _llm_type(self) -> str:
        return "lm_studio"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        try:
            payload = {
                "model": self.model_name,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": self.temperature,
                "max_tokens": self.max_tokens,
                "stream": False
            }
            
            if stop:
                payload["stop"] = stop
            
            response = requests.post(
                f"{self.base_url}/chat/completions",
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=30000
            )
            
            if response.status_code == 200:
                result = response.json()
                return result["choices"][0]["message"]["content"]
            else:
                return f"Error: LM Studio 응답 오류 (status: {response.status_code})"
                
        except Exception as e:
            return f"Error: LM Studio 통신 오류 - {str(e)}"


class LangChainTestCaseSystem:
    def __init__(self, 
                 persist_directory: str = "./chroma_db",
                 collection_name: str = "jira_test_cases",
                 embedding_model_name: str = "intfloat/multilingual-e5-large",
                 lm_studio_url: str = "http://127.0.0.1:1234/v1",
                 lm_studio_model: str = "qwen/qwen3-8b"):
        
        self.persist_directory = persist_directory
        self.collection_name = collection_name
        
        # 임베딩 모델 설정 (CPU 사용)
        model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
        encode_kwargs = {'normalize_embeddings': True, 'batch_size': 4}
        
        self.embeddings = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        
        # ChromaDB 연결
        self.vectorstore = self._connect_to_chroma()
        
        # LM Studio LLM 초기화
        self.llm = LMStudioLLM(
            base_url=lm_studio_url,
            model_name=lm_studio_model,
            temperature=0.1,
            max_tokens=2048
        )
        
        # 테스트케이스 찾기용 체인
        self.search_chain = self._setup_search_chain()
        
        # 테스트케이스 생성용 체인
        self.generation_chain = self._setup_generation_chain()
        
        # 결과 저장용
        self.test_results = []
    
    def _connect_to_chroma(self) -> Chroma:
        """기존 ChromaDB 컬렉션에 연결"""
        try:
            vectorstore = Chroma(
                collection_name=self.collection_name,
                embedding_function=self.embeddings,
                persist_directory=self.persist_directory
            )
            print(f"✅ 기존 ChromaDB 컬렉션 '{self.collection_name}' 연결 완료")
            return vectorstore
        except Exception as e:
            print(f"❌ ChromaDB 연결 실패: {e}")
            raise
    
    def _setup_search_chain(self) -> RetrievalQA:
        """테스트케이스 검색용 체인 설정"""
        search_prompt_template = PromptTemplate(
            template="""당신은 테스트케이스 검색 전문가입니다. 사용자의 요청에 맞는 테스트케이스를 찾아서 정리해주세요.

검색된 테스트케이스들:
{context}

사용자 요청: {question}

답변 가이드라인:
1. 요청과 관련된 테스트케이스들을 명확하게 정리해주세요
2. 각 테스트케이스의 이슈 키, 테스트 데이터, 테스트 스텝, 예상 결과를 포함해주세요
3. 이슈별로 그룹핑하여 체계적으로 보여주세요
4. 찾은 테스트케이스가 충분하지 않다면 추가 검색 키워드를 제안해주세요

답변:""",
            input_variables=["context", "question"]
        )
        
        return RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={"score_threshold": 0.6, "k": 10}
            ),
            chain_type_kwargs={"prompt": search_prompt_template},
            return_source_documents=True
        )
    
    def _setup_generation_chain(self) -> RetrievalQA:
        """테스트케이스 생성용 체인 설정"""
        generation_prompt_template = PromptTemplate(
            template="""당신은 테스트케이스 작성 전문가입니다. 기존 테스트케이스들을 참고하여 새로운 테스트케이스를 작성해주세요.

참고할 기존 테스트케이스들:
{context}

사용자 요청: {question}

새로운 테스트케이스 작성 가이드라인:
1. 사용자 요청에 맞는 구체적인 테스트케이스를 작성해주세요
2. 테스트 목적, 전제 조건, 테스트 스텝, 예상 결과를 명확히 구분해주세요
3. 기존 테스트케이스의 패턴과 형식을 참고하되, 새로운 시나리오를 제안해주세요
4. 테스트 데이터는 구체적이고 현실적으로 작성해주세요
5. Edge Case나 예외 상황도 고려해주세요
6. 가능하다면 여러 개의 테스트케이스 시나리오를 제안해주세요

테스트케이스 형식:
**테스트 케이스: [테스트 제목]**
- **테스트 목적**: [목적 설명]
- **전제 조건**: [사전 조건]
- **테스트 스텝**:
  1. [스텝 1]
  2. [스텝 2]
  ...
- **테스트 데이터**: [필요한 데이터]
- **예상 결과**: [기대하는 결과]

답변:""",
            input_variables=["context", "question"]
        )
        
        return RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 8}
            ),
            chain_type_kwargs={"prompt": generation_prompt_template},
            return_source_documents=True
        )
    
    def find_test_cases(self, query: str, filters: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """테스트케이스 찾기 기능"""
        try:
            print(f"🔍 테스트케이스 검색 중: {query}")
            
            formatted_query = f"query: {query}"
            response = self.search_chain({"query": formatted_query})
            
            # 소스 문서 정리
            source_docs = []
            for doc in response.get("source_documents", []):
                source_docs.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata,
                    "issue_key": doc.metadata.get("issue_key", ""),
                    "step_index": doc.metadata.get("step_index", "")
                })
            
            result = {
                "query": query,
                "answer": response["result"],
                "found_test_cases": source_docs,
                "total_found": len(source_docs)
            }
            
            # 결과 저장
            self._save_result("find_test_cases", query, result)
            
            return result
            
        except Exception as e:
            error_msg = f"테스트케이스 검색 중 오류: {str(e)}"
            print(f"❌ {error_msg}")
            error_result = {"query": query, "error": error_msg}
            self._save_result("find_test_cases", query, error_result)
            return error_result
    
    def generate_test_case(self, requirement: str) -> Dict[str, Any]:
        """테스트케이스 생성 기능"""
        try:
            print(f"🚀 테스트케이스 생성 중: {requirement}")
            
            formatted_requirement = f"query: {requirement}"
            response = self.generation_chain({"query": formatted_requirement})
            
            # 참고한 소스 문서들
            reference_docs = []
            for doc in response.get("source_documents", []):
                reference_docs.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata,
                    "issue_key": doc.metadata.get("issue_key", ""),
                    "step_index": doc.metadata.get("step_index", "")
                })
            
            result = {
                "requirement": requirement,
                "generated_test_case": response["result"],
                "reference_test_cases": reference_docs,
                "reference_count": len(reference_docs)
            }
            
            # 결과 저장
            self._save_result("generate_test_case", requirement, result)
            
            return result
            
        except Exception as e:
            error_msg = f"테스트케이스 생성 중 오류: {str(e)}"
            print(f"❌ {error_msg}")
            error_result = {"requirement": requirement, "error": error_msg}
            self._save_result("generate_test_case", requirement, error_result)
            return error_result
    
    def _save_result(self, test_type: str, query: str, result: Any):
        """결과 저장"""
        self.test_results.append({
            "timestamp": datetime.datetime.now().isoformat(),
            "framework": "langchain",
            "test_type": test_type,
            "query": query,
            "result": result
        })
    
    def save_results_to_file(self, filename: str = None):
        """결과를 파일에 저장"""
        if filename is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"langchain_testcase_results_{timestamp}.json"
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.test_results, f, ensure_ascii=False, indent=2)
            print(f"✅ LangChain 결과가 {filename}에 저장되었습니다.")
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
    
    def print_find_result(self, result: Dict[str, Any]):
        """테스트케이스 찾기 결과 출력"""
        print(f"\n📋 검색 결과:")
        print("=" * 50)
        if "error" in result:
            print(f"❌ 오류: {result['error']}")
            return
        
        print(f"🔍 검색 쿼리: {result['query']}")
        print(f"📊 찾은 테스트케이스 수: {result['total_found']}개")
        print(f"\n💡 분석 결과:")
        print(result['answer'])
        
        if result['found_test_cases']:
            print(f"\n📚 찾은 테스트케이스들:")
            for i, doc in enumerate(result['found_test_cases'][:5], 1):
                issue_key = doc['issue_key']
                step_idx = doc['step_index']
                print(f"\n{i}. {issue_key}_step_{step_idx}")
                print(f"   {doc['content'][:150]}...")
    
    def print_generation_result(self, result: Dict[str, Any]):
        """테스트케이스 생성 결과 출력"""
        print(f"\n🚀 생성 결과:")
        print("=" * 50)
        if "error" in result:
            print(f"❌ 오류: {result['error']}")
            return
        
        print(f"📝 요구사항: {result['requirement']}")
        print(f"📊 참고한 테스트케이스 수: {result['reference_count']}개")
        print(f"\n🎯 생성된 테스트케이스:")
        print(result['generated_test_case'])
    
    def run_comprehensive_test(self):
        """종합 테스트 실행"""
        print("🚀 LangChain 테스트케이스 시스템 종합 테스트 시작")
        print("=" * 60)
        
        # 1. 테스트케이스 찾기 테스트들
        find_queries = [
            "Master Admin과 관련된 테스트케이스 중에서 master admin 설정 갯수에 대한 테스트케이스를 가져와줘"
        ]
        
        print("\n🔍 테스트케이스 찾기 테스트")
        print("-" * 40)
        for query in find_queries:
            result = self.find_test_cases(query)
            self.print_find_result(result)
            print("\n" + "="*30 + "\n")
        
        # 2. 테스트케이스 생성 테스트들
        generation_requirements = [
            "장치에 전체 관리자 설정을 강제할 수 있는 Master Admin 기능이 있는데 이 기능은 다음과 같이 동작을 해. 다만 이 기능이 동작을 하기 위해서는 조건이 있어. 버전이 V1.4.0 이상으로 생산된 제품이어야해. 조건에 부합되는 장치의 전원이 인가되면 화면에 Master Admin 설정화면이 표시가 돼. 하지만 버전이 V1.4.0 이하로 생산된 제품의 경우에는 장치 전원이 인가되면 메인화면이 표시가 돼. 버전은 BS3의 이전 버전들을 참고해서 테스트 케이스로 작성해줘."
        ]
        
        print("\n🚀 테스트케이스 생성 테스트")
        print("-" * 40)
        for requirement in generation_requirements:
            result = self.generate_test_case(requirement)
            self.print_generation_result(result)
            print("\n" + "="*30 + "\n")
        
        # 결과 저장
        self.save_results_to_file()
        
        print(f"\n🎉 LangChain 종합 테스트 완료!")
        print(f"총 {len(self.test_results)}개의 결과가 저장되었습니다.")


# 사용 예시
if __name__ == "__main__":
    # LangChain 테스트케이스 시스템 초기화
    langchain_system = LangChainTestCaseSystem(
        lm_studio_url="http://127.0.0.1:1234/v1",
        lm_studio_model="qwen/qwen3-8b"
    )
    
    # 종합 테스트 실행
    langchain_system.run_comprehensive_test()

✅ 기존 ChromaDB 컬렉션 'jira_test_cases' 연결 완료
✅ LM Studio 연결 성공! 사용 가능한 모델: 2개
🚀 LangChain 테스트케이스 시스템 종합 테스트 시작

🔍 테스트케이스 찾기 테스트
----------------------------------------
🔍 테스트케이스 검색 중: Master Admin과 관련된 테스트케이스 중에서 master admin 설정 갯수에 대한 테스트케이스를 가져와줘

📋 검색 결과:
🔍 검색 쿼리: Master Admin과 관련된 테스트케이스 중에서 master admin 설정 갯수에 대한 테스트케이스를 가져와줘
📊 찾은 테스트케이스 수: 10개

💡 분석 결과:
Error: LM Studio 통신 오류 - HTTPConnectionPool(host='127.0.0.1', port=1234): Read timed out. (read timeout=120)

📚 찾은 테스트케이스들:

1. COMMONR-380_step_1
   Test Step: 1. Device> 관리자 설정
Test Data: 1. Master Password 설정
2. 전체 관리자 설정
3. 장치 설정 관리자 설정
4. 사용자 관리자 설정
Expected Result: 1. 사용자가 선택한 관리자로 설정되어야 한다.
>...

2. COMMONR-218_step_5
   Test Step: [Master Admin 지원 신규HW]
1. Master Admin 설정
2. Secure Tamper: Enable 설정
3. Secure Tamper 발생
Test Data: UI 지원모델

※ Device> Device Info> Mac을 길...

3. COMMONR-247_step_1
   Test Step: 1. 장치 두대이상 선택> Batch Edit 클릭
2. 다수의 관리자(All/User/Config) 최대로 설정 후 적용
3. 장치상세정보창 진입
4. 관리자 확인
5. All/User/Config 관리자로 메뉴진

### 📌 5-2. llamaindex을 이용한 구축 with LM Studio
> 🔗 llamaindex을 이용해서 저장한 임베딩 값을 Langchain, LM Studio와 연계하여 최대의 값 도출
- llamaindex을 이용하여 chroma DB 결과값 QA RAG로 구현
- 해당 RAG LM Studio LLM과 연동하여 기대값에 대한 결과값 도출

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.llms import CustomLLM, CompletionResponse, LLMMetadata
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor, LLMRerank
from llama_index.core.prompts import PromptTemplate
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from typing import List, Dict, Any, Optional, Generator, Sequence
import json
import requests
import warnings
import datetime

# FutureWarning 무시
warnings.filterwarnings("ignore", category=FutureWarning)


class LMStudioLLM(CustomLLM):
    """LM Studio와 연동하는 LlamaIndex용 커스텀 LLM 클래스"""
    
    model_name: str = "qwen/qwen3-8b"
    base_url: str = "http://127.0.0.1:1234/v1"
    temperature: float = 0.1
    max_tokens: int = 2048
    
    def __init__(self, 
                 base_url: str = "http://127.0.0.1:1234/v1",
                 model_name: str = "qwen/qwen3-8b",
                 temperature: float = 0.1,
                 max_tokens: int = 2048,
                 **kwargs):
        super().__init__(**kwargs)
        self.base_url = base_url.rstrip('/')
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self._test_connection()
    
    def _test_connection(self):
        """LM Studio 연결 테스트"""
        try:
            response = requests.get(f"{self.base_url}/models", timeout=5)
            if response.status_code == 200:
                models = response.json()
                print(f"✅ LM Studio 연결 성공! 사용 가능한 모델: {len(models.get('data', []))}개")
            else:
                print(f"⚠️ LM Studio 연결 상태 확인 필요: {response.status_code}")
        except Exception as e:
            print(f"❌ LM Studio 연결 실패: {e}")
    
    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=8192,
            num_output=self.max_tokens,
            model_name=self.model_name,
        )
    
    def complete(self, prompt: str, **kwargs) -> CompletionResponse:
        try:
            payload = {
                "model": self.model_name,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": self.temperature,
                "max_tokens": self.max_tokens,
                "stream": False
            }
            
            response = requests.post(
                f"{self.base_url}/chat/completions",
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=30000
            )
            
            if response.status_code == 200:
                result = response.json()
                text = result["choices"][0]["message"]["content"]
                return CompletionResponse(text=text)
            else:
                error_text = f"Error: LM Studio 응답 오류 (status: {response.status_code})"
                return CompletionResponse(text=error_text)
                
        except Exception as e:
            error_text = f"Error: LM Studio 통신 오류 - {str(e)}"
            return CompletionResponse(text=error_text)
    
    def chat(self, messages: Sequence[ChatMessage], **kwargs) -> CompletionResponse:
        formatted_messages = []
        for msg in messages:
            formatted_messages.append({
                "role": msg.role.value if hasattr(msg.role, 'value') else str(msg.role),
                "content": msg.content
            })
        
        try:
            payload = {
                "model": self.model_name,
                "messages": formatted_messages,
                "temperature": self.temperature,
                "max_tokens": self.max_tokens,
                "stream": False
            }
            
            response = requests.post(
                f"{self.base_url}/chat/completions",
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=30000
            )
            
            if response.status_code == 200:
                result = response.json()
                text = result["choices"][0]["message"]["content"]
                return CompletionResponse(text=text)
            else:
                error_text = f"Error: LM Studio 응답 오류 (status: {response.status_code})"
                return CompletionResponse(text=error_text)
                
        except Exception as e:
            error_text = f"Error: LM Studio 통신 오류 - {str(e)}"
            return CompletionResponse(text=error_text)
    
    def stream_complete(self, prompt: str, **kwargs) -> Generator[CompletionResponse, None, None]:
        response = self.complete(prompt, **kwargs)
        yield response
    
    def stream_chat(self, messages: Sequence[ChatMessage], **kwargs) -> Generator[CompletionResponse, None, None]:
        response = self.chat(messages, **kwargs)
        yield response


class LlamaIndexTestCaseSystem:
    def __init__(self, 
                 persist_directory: str = "./chroma_db",
                 collection_name: str = "jira_test_cases",
                 embedding_model_name: str = "intfloat/multilingual-e5-large",
                 lm_studio_url: str = "http://127.0.0.1:1234/v1",
                 lm_studio_model: str = "qwen/qwen3-8b"):
        
        self.persist_directory = persist_directory
        self.collection_name = collection_name
        self.lm_studio_url = lm_studio_url
        self.lm_studio_model = lm_studio_model
        
        # Global Settings 설정
        self._setup_global_settings(embedding_model_name)
        
        # ChromaDB 벡터 스토어 연결
        self.vector_store = self._connect_to_chroma()
        
        # 인덱스 생성
        self.index = self._create_index()
        
        # 테스트케이스 찾기용 쿼리 엔진
        self.search_engine = self._setup_search_engine()
        
        # 테스트케이스 생성용 쿼리 엔진
        self.generation_engine = self._setup_generation_engine()
        
        # 결과 저장용
        self.test_results = []
    
    def _setup_global_settings(self, embedding_model_name: str):
        """LlamaIndex Global Settings 설정"""
        # 임베딩 모델 설정 (CPU 사용)
        Settings.embed_model = HuggingFaceEmbedding(
            model_name=embedding_model_name,
            device="cpu",
            trust_remote_code=True,
            normalize=True
        )
        
        # LM Studio LLM 설정
        Settings.llm = LMStudioLLM(
            base_url=self.lm_studio_url,
            model_name=self.lm_studio_model,
            temperature=0.1,
            max_tokens=2048
        )
        
        print(f"✅ LM Studio LLM 설정 완료: {self.lm_studio_url}")
    
    def _connect_to_chroma(self) -> ChromaVectorStore:
        """기존 ChromaDB 컬렉션에 연결"""
        try:
            chroma_client = chromadb.PersistentClient(path=self.persist_directory)
            chroma_collection = chroma_client.get_collection(name=self.collection_name)
            vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
            
            print(f"✅ 기존 ChromaDB 컬렉션 '{self.collection_name}' 연결 완료")
            return vector_store
            
        except Exception as e:
            print(f"❌ ChromaDB 연결 실패: {e}")
            raise
    
    def _create_index(self) -> VectorStoreIndex:
        """기존 벡터 스토어로부터 인덱스 생성"""
        try:
            storage_context = StorageContext.from_defaults(vector_store=self.vector_store)
            index = VectorStoreIndex.from_vector_store(
                vector_store=self.vector_store,
                storage_context=storage_context
            )
            
            print("✅ LlamaIndex 벡터 인덱스 생성 완료")
            return index
            
        except Exception as e:
            print(f"❌ 인덱스 생성 실패: {e}")
            raise
    
    def _setup_search_engine(self) -> RetrieverQueryEngine:
        """테스트케이스 검색용 쿼리 엔진 설정"""
        # 검색에 특화된 리트리버 (더 많은 결과, 낮은 임계값)
        retriever = VectorIndexRetriever(
            index=self.index,
            similarity_top_k=12
        )
        
        # 검색용 포스트프로세서 (관련성 높은 결과만 필터링)
        postprocessors = [
            SimilarityPostprocessor(similarity_cutoff=0.6)
        ]
        
        query_engine = RetrieverQueryEngine(
            retriever=retriever,
            node_postprocessors=postprocessors
        )
        
        # 검색용 커스텀 프롬프트
        search_prompt = PromptTemplate(
            """당신은 테스트케이스 검색 및 분석 전문가입니다. 사용자의 요청에 맞는 테스트케이스를 찾아서 체계적으로 정리해주세요.

검색된 테스트케이스들:
{context_str}

사용자 검색 요청: {query_str}

검색 결과 분석 가이드라인:
1. 요청과 관련된 테스트케이스들을 명확하게 정리해주세요
2. 각 테스트케이스의 이슈 키, 테스트 데이터, 테스트 스텝, 예상 결과를 포함해주세요
3. 이슈별로 그룹핑하여 체계적으로 보여주세요
4. 찾은 테스트케이스가 충분하지 않다면 추가 검색 키워드를 제안해주세요

답변:"""
        )
        
        query_engine.update_prompts({"response_synthesizer:text_qa_template": search_prompt})
        
        print("✅ 검색용 쿼리 엔진 설정 완료")
        return query_engine
    
    def _setup_generation_engine(self) -> RetrieverQueryEngine:
        """테스트케이스 생성용 쿼리 엔진 설정"""
        # 생성에 특화된 리트리버 (적은 수의 고품질 예시)
        retriever = VectorIndexRetriever(
            index=self.index,
            similarity_top_k=8
        )
        
        # 생성용 포스트프로세서 (높은 품질의 참고 자료만 선별)
        postprocessors = [
            SimilarityPostprocessor(similarity_cutoff=0.7)
        ]
        
        query_engine = RetrieverQueryEngine(
            retriever=retriever,
            node_postprocessors=postprocessors
        )
        
        # 생성용 커스텀 프롬프트
        generation_prompt = PromptTemplate(
            """당신은 테스트케이스 작성 전문가입니다. 기존 테스트케이스들의 패턴과 구조를 분석하여 새로운 고품질 테스트케이스를 작성해주세요.

참고할 기존 테스트케이스들:
{context_str}

테스트케이스 작성 요청: {query_str}

새로운 테스트케이스 작성 가이드라인:
1. 사용자 요청에 맞는 구체적인 테스트케이스를 작성해주세요
2. 테스트 목적, 전제 조건, 테스트 스텝, 예상 결과를 명확히 구분해주세요
3. 기존 테스트케이스의 패턴과 형식을 참고하되, 새로운 시나리오를 제안해주세요
4. 테스트 데이터는 구체적이고 현실적으로 작성해주세요
5. Edge Case나 예외 상황도 고려해주세요
6. 가능하다면 여러 개의 테스트케이스 시나리오를 제안해주세요

테스트케이스 형식:
**테스트 케이스: [테스트 제목]**
- **테스트 목적**: [목적 설명]
- **전제 조건**: [사전 조건]
- **테스트 스텝**:
  1. [스텝 1]
  2. [스텝 2]
  ...
- **테스트 데이터**: [필요한 데이터]
- **예상 결과**: [기대하는 결과]

답변:"""
        )
        
        query_engine.update_prompts({"response_synthesizer:text_qa_template": generation_prompt})
        
        print("✅ 생성용 쿼리 엔진 설정 완료")
        return query_engine
    
    def find_test_cases(self, query: str, use_advanced_search: bool = True) -> Dict[str, Any]:
        """고급 테스트케이스 검색 기능"""
        try:
            print(f"🔍 LlamaIndex 고급 검색 중: {query}")
            
            formatted_query = f"query: {query}"
            
            if use_advanced_search:
                # 고급 검색: 다단계 검색 및 재랭킹
                
                # 1단계: 넓은 범위 검색
                broad_retriever = VectorIndexRetriever(
                    index=self.index,
                    similarity_top_k=20
                )
                broad_nodes = broad_retriever.retrieve(formatted_query)
                
                # 2단계: 관련성 기반 필터링
                similarity_processor = SimilarityPostprocessor(similarity_cutoff=0.5)
                filtered_nodes = similarity_processor.postprocess_nodes(broad_nodes)
                
                # 검색 결과를 직접 처리
                search_result = self._process_search_nodes(filtered_nodes, query)
                
                # LLM을 통한 분석
                analysis_response = self.search_engine.query(formatted_query)
                search_result["llm_analysis"] = str(analysis_response)
                
            else:
                # 기본 검색
                response = self.search_engine.query(formatted_query)
                search_result = {
                    "query": query,
                    "llm_analysis": str(response),
                    "found_nodes": []
                }
                
                if hasattr(response, 'source_nodes'):
                    search_result["found_nodes"] = self._process_search_nodes(response.source_nodes, query)["found_nodes"]
            
            # 결과 저장
            self._save_result("find_test_cases", query, search_result)
            
            return search_result
            
        except Exception as e:
            error_msg = f"테스트케이스 검색 중 오류: {str(e)}"
            print(f"❌ {error_msg}")
            error_result = {"query": query, "error": error_msg}
            self._save_result("find_test_cases", query, error_result)
            return error_result
    
    def generate_test_case(self, requirement: str, use_advanced_generation: bool = True) -> Dict[str, Any]:
        """고급 테스트케이스 생성 기능"""
        try:
            print(f"🚀 LlamaIndex 고급 생성 중: {requirement}")
            
            formatted_requirement = f"query: {requirement}"
            
            if use_advanced_generation:
                # 고급 생성: 참조 자료 품질 최적화
                
                # 1단계: 관련 테스트케이스 검색
                reference_retriever = VectorIndexRetriever(
                    index=self.index,
                    similarity_top_k=15
                )
                reference_nodes = reference_retriever.retrieve(formatted_requirement)
                
                # 2단계: 고품질 참조 자료만 선별
                quality_processor = SimilarityPostprocessor(similarity_cutoff=0.8)
                quality_nodes = quality_processor.postprocess_nodes(reference_nodes)
                
                # 참조 자료 분석
                reference_analysis = self._analyze_reference_nodes(quality_nodes)
                
                # LLM을 통한 테스트케이스 생성
                generation_response = self.generation_engine.query(formatted_requirement)
                
                generation_result = {
                    "requirement": requirement,
                    "generated_test_case": str(generation_response),
                    "reference_analysis": reference_analysis,
                    "reference_count": len(quality_nodes),
                    "generation_quality_score": self._calculate_generation_quality(str(generation_response))
                }
                
            else:
                # 기본 생성
                response = self.generation_engine.query(formatted_requirement)
                generation_result = {
                    "requirement": requirement,
                    "generated_test_case": str(response),
                    "reference_count": 0
                }
            
            # 결과 저장
            self._save_result("generate_test_case", requirement, generation_result)
            
            return generation_result
            
        except Exception as e:
            error_msg = f"테스트케이스 생성 중 오류: {str(e)}"
            print(f"❌ {error_msg}")
            error_result = {"requirement": requirement, "error": error_msg}
            self._save_result("generate_test_case", requirement, error_result)
            return error_result
    
    def _process_search_nodes(self, nodes, query):
        """검색 노드들을 처리하여 구조화된 결과 생성"""
        found_nodes = []
        issue_groups = {}
        
        for node in nodes:
            node_info = {
                "content": node.text,
                "metadata": node.metadata,
                "score": float(node.score) if node.score else 0.0,
                "issue_key": node.metadata.get("issue_key", ""),
                "step_index": node.metadata.get("step_index", "")
            }
            found_nodes.append(node_info)
            
            # 이슈별 그룹핑
            issue_key = node.metadata.get("issue_key", "UNKNOWN")
            if issue_key not in issue_groups:
                issue_groups[issue_key] = []
            issue_groups[issue_key].append(node_info)
        
        # 이슈별 정렬
        for issue_key in issue_groups:
            issue_groups[issue_key].sort(
                key=lambda x: int(x["step_index"]) if x["step_index"].isdigit() else 0
            )
        
        return {
            "query": query,
            "found_nodes": found_nodes,
            "issue_groups": issue_groups,
            "total_found": len(found_nodes),
            "unique_issues": len(issue_groups)
        }
    
    def _analyze_reference_nodes(self, nodes):
        """참조 노드들을 분석하여 패턴 추출"""
        if not nodes:
            return {"pattern_analysis": "참조 자료가 충분하지 않습니다."}
        
        patterns = {
            "common_steps": [],
            "test_data_patterns": [],
            "result_patterns": [],
            "issue_types": set()
        }
        
        for node in nodes:
            # 이슈 타입 수집
            issue_key = node.metadata.get("issue_key", "")
            if issue_key:
                patterns["issue_types"].add(issue_key.split("-")[0] if "-" in issue_key else issue_key)
            
            # 텍스트 패턴 분석 (간단한 키워드 기반)
            content = node.text.lower()
            if "step" in content:
                patterns["common_steps"].append(content[:100])
            if "data" in content:
                patterns["test_data_patterns"].append(content[:100])
            if "result" in content or "expected" in content:
                patterns["result_patterns"].append(content[:100])
        
        patterns["issue_types"] = list(patterns["issue_types"])
        
        return {
            "pattern_analysis": f"분석된 {len(nodes)}개 참조 자료에서 {len(patterns['issue_types'])}개 이슈 타입 발견",
            "patterns": patterns,
            "reference_quality": len(nodes)
        }
    
    def _calculate_generation_quality(self, generated_text):
        """생성된 테스트케이스의 품질 점수 계산 (간단한 휴리스틱)"""
        quality_score = 0
        
        # 기본 구조 확인
        if "테스트" in generated_text:
            quality_score += 10
        if "스텝" in generated_text or "단계" in generated_text:
            quality_score += 15
        if "예상" in generated_text or "결과" in generated_text:
            quality_score += 15
        if "데이터" in generated_text:
            quality_score += 10
        if "조건" in generated_text:
            quality_score += 10
        
        # 길이 기반 점수 (너무 짧거나 길지 않은 적절한 길이)
        text_length = len(generated_text)
        if 200 <= text_length <= 2000:
            quality_score += 20
        elif text_length > 100:
            quality_score += 10
        
        # 구체성 점수 (숫자나 구체적인 용어 포함)
        import re
        if re.search(r'\d+', generated_text):
            quality_score += 10
        if len(re.findall(r'[가-힣]{2,}', generated_text)) > 10:
            quality_score += 10
        
        return min(quality_score, 100)  # 최대 100점
    
    def _save_result(self, test_type: str, query: str, result: Any):
        """결과 저장"""
        self.test_results.append({
            "timestamp": datetime.datetime.now().isoformat(),
            "framework": "llamaindex",
            "test_type": test_type,
            "query": query,
            "result": result
        })
    
    def save_results_to_file(self, filename: str = None):
        """결과를 파일에 저장"""
        if filename is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"llamaindex_testcase_results_{timestamp}.json"
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.test_results, f, ensure_ascii=False, indent=2)
            print(f"✅ LlamaIndex 결과가 {filename}에 저장되었습니다.")
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
    
    def print_find_result(self, result: Dict[str, Any]):
        """테스트케이스 찾기 결과 출력"""
        print(f"\n🔍 LlamaIndex 검색 결과:")
        print("=" * 50)
        
        if "error" in result:
            print(f"❌ 오류: {result['error']}")
            return
        
        print(f"📝 검색 쿼리: {result['query']}")
        
        if "found_nodes" in result:
            print(f"📊 찾은 테스트케이스: {result.get('total_found', 0)}개")
            print(f"🏷️ 관련 이슈: {result.get('unique_issues', 0)}개")
        
        if "llm_analysis" in result:
            print(f"\n🤖 LLM 분석 결과:")
            print(result['llm_analysis'])
        
        if "issue_groups" in result:
            print(f"\n📚 이슈별 테스트케이스:")
            for issue_key, nodes in list(result['issue_groups'].items())[:3]:  # 상위 3개만 출력
                print(f"\n🔗 {issue_key}:")
                for i, node in enumerate(nodes[:2], 1):  # 각 이슈당 2개까지만
                    score = node.get('score', 0)
                    print(f"  {i}. (점수: {score:.3f}) {node['content'][:120]}...")
    
    def print_generation_result(self, result: Dict[str, Any]):
        """테스트케이스 생성 결과 출력"""
        print(f"\n🚀 LlamaIndex 생성 결과:")
        print("=" * 50)
        
        if "error" in result:
            print(f"❌ 오류: {result['error']}")
            return
        
        print(f"📋 요구사항: {result['requirement']}")
        print(f"📊 참조 자료: {result.get('reference_count', 0)}개")
        
        if "generation_quality_score" in result:
            quality = result['generation_quality_score']
            print(f"⭐ 생성 품질 점수: {quality}/100")
        
        if "reference_analysis" in result:
            ref_analysis = result['reference_analysis']
            print(f"🔍 참조 분석: {ref_analysis.get('pattern_analysis', 'N/A')}")
        
        print(f"\n🎯 생성된 테스트케이스:")
        print("-" * 40)
        print(result['generated_test_case'])
    
    def run_comprehensive_test(self):
        """LlamaIndex 종합 테스트 실행"""
        print("🚀 LlamaIndex 테스트케이스 시스템 종합 테스트 시작")
        print("=" * 60)
        
        # 1. 고급 테스트케이스 찾기 테스트들
        find_queries = [
            "Master Admin과 관련된 테스트케이스 중에서 master admin 설정 갯수에 대한 테스트케이스를 가져와줘"
        ]
        
        print("\n🔍 고급 테스트케이스 검색 테스트")
        print("-" * 40)
        for query in find_queries:
            result = self.find_test_cases(query, use_advanced_search=True)
            self.print_find_result(result)
            print("\n" + "="*30 + "\n")
        
        # 2. 고급 테스트케이스 생성 테스트들
        generation_requirements = [
            "장치에 전체 관리자 설정을 강제할 수 있는 Master Admin 기능이 있는데 이 기능은 다음과 같이 동작을 해. 다만 이 기능이 동작을 하기 위해서는 조건이 있어. 버전이 V1.4.0 이상으로 생산된 제품이어야해. 조건에 부합되는 장치의 전원이 인가되면 화면에 Master Admin 설정화면이 표시가 돼. 하지만 버전이 V1.4.0 이하로 생산된 제품의 경우에는 장치 전원이 인가되면 메인화면이 표시가 돼. 버전은 BS3의 이전 버전들을 참고해서 테스트 케이스로 작성해줘."
        ]
        
        print("\n🚀 고급 테스트케이스 생성 테스트")
        print("-" * 40)
        for requirement in generation_requirements:
            result = self.generate_test_case(requirement, use_advanced_generation=True)
            self.print_generation_result(result)
            print("\n" + "="*30 + "\n")
        
        # 결과 저장
        self.save_results_to_file()
        
        print(f"\n🎉 LlamaIndex 종합 테스트 완료!")
        print(f"총 {len(self.test_results)}개의 결과가 저장되었습니다.")


# 사용 예시
if __name__ == "__main__":
    # LlamaIndex 테스트케이스 시스템 초기화
    llamaindex_system = LlamaIndexTestCaseSystem(
        lm_studio_url="http://127.0.0.1:1234/v1",
        lm_studio_model="qwen/qwen3-8b"
    )
    
    # 종합 테스트 실행
    llamaindex_system.run_comprehensive_test()

✅ LM Studio 연결 성공! 사용 가능한 모델: 2개
✅ LM Studio LLM 설정 완료: http://127.0.0.1:1234/v1
✅ 기존 ChromaDB 컬렉션 'jira_test_cases' 연결 완료
✅ LlamaIndex 벡터 인덱스 생성 완료
✅ 검색용 쿼리 엔진 설정 완료
✅ 생성용 쿼리 엔진 설정 완료
🚀 LlamaIndex 테스트케이스 시스템 종합 테스트 시작

🔍 고급 테스트케이스 검색 테스트
----------------------------------------
🔍 LlamaIndex 고급 검색 중: Master Admin과 관련된 테스트케이스 중에서 master admin 설정 갯수에 대한 테스트케이스를 가져와줘

🔍 LlamaIndex 검색 결과:
📝 검색 쿼리: Master Admin과 관련된 테스트케이스 중에서 master admin 설정 갯수에 대한 테스트케이스를 가져와줘
📊 찾은 테스트케이스: 20개
🏷️ 관련 이슈: 11개

🤖 LLM 분석 결과:
Error: LM Studio 통신 오류 - HTTPConnectionPool(host='127.0.0.1', port=1234): Read timed out. (read timeout=120)

📚 이슈별 테스트케이스:

🔗 COMMONR-380:
  1. (점수: 0.892) Test Step: 1. Device> 관리자 설정
Test Data: 1. Master Password 설정
2. 전체 관리자 설정
3. 장치 설정 관리자 설정
4. 사용자 관리자 설정
Expected Result...
  2. (점수: 0.861) Test Step: 1. 관리자 설정/미설정
2. 메뉴 진입 상태 > 인증 
3. 메뉴 진입 시도와 동시에 인증 시도
Expected Result: 1. 설정된 권한으로 출입문제어가 되어야 한다.
> 메뉴 진입 후 ...

🔗 COMMONR-218:
  1. (점수: 0.872) Test Step: [Master Admin 지원 신규HW]
1. M

# LLAMAINDEX WITH LANGCHAIN
# Test
# Test
# Test
# Test


In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex, Document, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.llms import CustomLLM, CompletionResponse, LLMMetadata
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.prompts import PromptTemplate
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.vector_stores import ExactMatchFilter, MetadataFilters, FilterOperator

# LangChain 관련 임포트
from langchain_community.llms import OpenAI # LM Studio를 OpenAI API 호환으로 사용할 경우
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate as LCPromptTemplate # LangChain의 PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from llama_index.core.langchain_helpers import LlamaIndexRetriever # LlamaIndex 리트리버를 LangChain에 통합

from typing import List, Dict, Any, Optional, Generator, Sequence
import json
import requests
import warnings

# LangChain DeprecationWarning 무시 (해결되기를 기다리며 임시 조치)
warnings.filterwarnings("ignore", category=DeprecationWarning, module='langchain_core._api.deprecation')


class LMStudioLLM(CustomLLM):
    """LM Studio와 연동하는 LlamaIndex용 커스텀 LLM 클래스"""
    
    model_name: str = "qwen/qwen3-8b"
    base_url: str = "http://127.0.0.1:1234/v1"
    temperature: float = 0.1
    max_tokens: int = 2048
    
    def __init__(self, 
                 base_url: str = "http://127.0.0.1:1234/v1",
                 model_name: str = "qwen/qwen3-8b",
                 temperature: float = 0.1,
                 max_tokens: int = 2048,
                 **kwargs):
        super().__init__(**kwargs)
        self.base_url = base_url.rstrip('/')
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self._test_connection()
    
    def _test_connection(self):
        try:
            response = requests.get(f"{self.base_url}/models", timeout=100000)
            if response.status_code == 200:
                models = response.json()
                print(f"✅ LM Studio 연결 성공! 사용 가능한 모델: {len(models.get('data', []))}개")
            else:
                print(f"⚠️ LM Studio 연결 상태 확인 필요: {response.status_code}")
        except Exception as e:
            print(f"❌ LM Studio 연결 실패: {e}")
            print("LM Studio가 실행 중인지 확인해주세요.")
    
    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=8192,
            num_output=self.max_tokens,
            model_name=self.model_name,
        )
    
    def complete(self, prompt: str, **kwargs) -> CompletionResponse:
        try:
            payload = {
                "model": self.model_name,
                "messages": [
                    {"role": "user", "content": prompt}
                ],
                "temperature": self.temperature,
                "max_tokens": self.max_tokens,
                "stream": False
            }
            response = requests.post(
                f"{self.base_url}/chat/completions",
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=100000
            )
            
            if response.status_code == 200:
                result = response.json()
                text = result["choices"][0]["message"]["content"]
                return CompletionResponse(text=text)
            else:
                error_text = f"Error: LM Studio 응답 오류 (status: {response.status_code})"
                return CompletionResponse(text=error_text)
                
        except Exception as e:
            error_text = f"Error: LM Studio 통신 오류 - {str(e)}"
            return CompletionResponse(text=error_text)
    
    def chat(self, messages: Sequence[ChatMessage], **kwargs) -> CompletionResponse:
        formatted_messages = []
        for msg in messages:
            formatted_messages.append({
                "role": msg.role.value if hasattr(msg.role, 'value') else str(msg.role),
                "content": msg.content
            })
        
        try:
            payload = {
                "model": self.model_name,
                "messages": formatted_messages,
                "temperature": self.temperature,
                "max_tokens": self.max_tokens,
                "stream": False
            }
            response = requests.post(
                f"{self.base_url}/chat/completions",
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=60
            )
            
            if response.status_code == 200:
                result = response.json()
                text = result["choices"][0]["message"]["content"]
                return CompletionResponse(text=text)
            else:
                error_text = f"Error: LM Studio 응답 오류 (status: {response.status_code})"
                return CompletionResponse(text=error_text)
                
        except Exception as e:
            error_text = f"Error: LM Studio 통신 오류 - {str(e)}"
            return CompletionResponse(text=error_text)
    
    def stream_complete(self, prompt: str, **kwargs) -> Generator[CompletionResponse, None, None]:
        response = self.complete(prompt, **kwargs)
        yield response
    
    def stream_chat(self, messages: Sequence[ChatMessage], **kwargs) -> Generator[CompletionResponse, None, None]:
        response = self.chat(messages, **kwargs)
        yield response


class TestCaseRAGLlamaIndexLMStudio:
    def __init__(self, 
                 persist_directory: str = "./chroma_db",
                 collection_name: str = "jira_test_cases",
                 embedding_model_name: str = "intfloat/multilingual-e5-large",
                 lm_studio_url: str = "http://localhost:1234/v1",
                 lm_studio_model: str = "local-model"):
        self.persist_directory = persist_directory
        self.collection_name = collection_name
        self.lm_studio_url = lm_studio_url
        self.lm_studio_model = lm_studio_model
        
        self._setup_global_settings(embedding_model_name)
        self.vector_store = self._connect_to_chroma()
        self.index = self._create_index()
        self.query_engine = self._setup_query_engine()
        self._setup_custom_prompts()
    
    def _setup_global_settings(self, embedding_model_name: str):
        Settings.embed_model = HuggingFaceEmbedding(
            model_name=embedding_model_name,
            device="cpu",
            trust_remote_code=True,
            normalize=True,
            query_instruction="query:",  
            text_instruction="passage:", 
        )
        
        Settings.llm = LMStudioLLM(
            base_url=self.lm_studio_url,
            model_name=self.lm_studio_model,
            temperature=0.1,
            max_tokens=2048
        )
        print(f"✅ LM Studio LLM 설정 완료: {self.lm_studio_url}")
    
    def _connect_to_chroma(self) -> ChromaVectorStore:
        try:
            chroma_client = chromadb.PersistentClient(path=self.persist_directory)
            chroma_collection = chroma_client.get_or_create_collection(name=self.collection_name)
            vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
            print(f"✅ 기존 ChromaDB 컬렉션 '{self.collection_name}' 연결 완료")
            return vector_store
        except Exception as e:
            print(f"❌ ChromaDB 연결 실패: {e}")
            raise
    
    def _create_index(self) -> VectorStoreIndex:
        try:
            storage_context = StorageContext.from_defaults(vector_store=self.vector_store)
            index = VectorStoreIndex.from_vector_store(
                vector_store=self.vector_store,
                storage_context=storage_context
            )
            print("✅ LlamaIndex 벡터 인덱스 생성 완료")
            return index
        except Exception as e:
            print(f"❌ 인덱스 생성 실패: {e}")
            raise
    
    def _setup_query_engine(self) -> RetrieverQueryEngine:
        retriever = VectorIndexRetriever(
            index=self.index,
            similarity_top_k=8,
            vector_store_query_mode="default"
        )
        postprocessors = [
            SimilarityPostprocessor(similarity_cutoff=0.7)
        ]
        query_engine = RetrieverQueryEngine(
            retriever=retriever,
            node_postprocessors=postprocessors
        )
        print("✅ LM Studio 쿼리 엔진 설정 완료")
        return query_engine
    
    def _setup_custom_prompts(self):
        qa_prompt_template = PromptTemplate(
            """당신은 테스트케이스 전문가입니다. 주어진 테스트 스텝들을 바탕으로 사용자의 질문에 정확하고 유용한 답변을 제공해주세요.

참고할 테스트 스텝들:
{context_str}

사용자 질문: {query_str}

답변 가이드라인:
1. 관련된 테스트 스텝들을 명확하게 설명해주세요
2. 각 테스트 스텝의 목적과 예상 결과를 포함해주세요
3. 테스트 데이터가 있다면 함께 제시해주세요
4. 이슈 키별로 그룹핑하여 전체적인 테스트 흐름을 보여주세요
5. 찾은 정보가 충분하지 않다면 추가로 필요한 정보를 제안해주세요
6. 답변은 한국어로 제공해주세요

답변:"""
        )
        self.query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_template})
    
    def search_test_cases(self, 
                         query: str, 
                         filters: Optional[Dict[str, Any]] = None,
                         top_k: int = 10) -> List[Dict[str, Any]]:
        try:
            formatted_query = query 
            llama_filters = None
            if filters:
                llama_filters = MetadataFilters(
                    filters=[ExactMatchFilter(key=k, value=v) for k, v in filters.items()],
                    condition=FilterOperator.AND
                )

            retriever = VectorIndexRetriever(
                index=self.index,
                similarity_top_k=top_k,
                filters=llama_filters
            )
            
            nodes = retriever.retrieve(formatted_query)
            
            results = []
            for node in nodes:
                result = {
                    "content": node.text,
                    "metadata": node.metadata,
                    "similarity_score": float(node.score) if node.score else 0.0,
                    "node_id": node.node_id
                }
                results.append(result)
            return results
        except Exception as e:
            print(f"❌ 검색 중 오류 발생: {e}")
            return []
    
    def ask_question(self, question: str, filters: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        try:
            print(f"🤖 LM Studio로 질문 처리 중: {question}")
            formatted_question = question
            current_query_engine = self.query_engine
            
            if filters:
                llama_filters = MetadataFilters(
                    filters=[ExactMatchFilter(key=k, value=v) for k, v in filters.items()],
                    condition=FilterOperator.AND
                )

                retriever = VectorIndexRetriever(
                    index=self.index,
                    similarity_top_k=8,
                    filters=llama_filters
                )
                postprocessors = [SimilarityPostprocessor(similarity_cutoff=0.7)]
                query_engine = RetrieverQueryEngine(
                    retriever=retriever,
                    node_postprocessors=postprocessors
                )
                qa_prompt_template = PromptTemplate(
                    """당신은 테스트케이스 전문가입니다. 주어진 테스트 스텝들을 바탕으로 사용자의 질문에 정확하고 유용한 답변을 제공해주세요.

참고할 테스트 스텝들:
{context_str}

사용자 질문: {query_str}

답변은 한국어로 제공해주세요.

답변:"""
                )
                query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_template})
                current_query_engine = query_engine
            
            response = current_query_engine.query(formatted_question)
            
            source_nodes = []
            if hasattr(response, 'source_nodes'):
                for node in response.source_nodes:
                    source_nodes.append({
                        "content": node.text,
                        "metadata": node.metadata,
                        "score": float(node.score) if node.score else 0.0
                    })
            
            grouped_sources = self.group_results_by_issue(source_nodes)
            
            return {
                "answer": str(response),
                "source_nodes": source_nodes,
                "grouped_sources": grouped_sources,
                "response_metadata": response.metadata if hasattr(response, 'metadata') else {}
            }
            
        except Exception as e:
            print(f"❌ 질문 처리 중 오류 발생: {e}")
            return {
                "answer": "죄송합니다. 질문을 처리하는 중 오류가 발생했습니다.",
                "source_nodes": [],
                "grouped_sources": {},
                "response_metadata": {}
            }
    
    def group_results_by_issue(self, results: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
        grouped = {}
        for result in results:
            issue_key = result["metadata"].get("issue_key", "UNKNOWN")
            if issue_key not in grouped:
                grouped[issue_key] = []
            grouped[issue_key].append(result)
        
        for issue_key in grouped:
            grouped[issue_key].sort(
                key=lambda x: int(x["metadata"].get("step_index", "0"))
            )
        return grouped
    
    def get_test_case_by_issue(self, issue_key: str) -> List[Dict[str, Any]]:
        filters = {"issue_key": issue_key}
        results = self.search_test_cases("", filters=filters, top_k=50) 
        results.sort(key=lambda x: int(x["metadata"].get("step_index", "0")))
        return results
    
    def create_custom_query_engine(self, 
                                  similarity_top_k: int = 8,
                                  similarity_cutoff: float = 0.7,
                                  filters: Optional[Dict[str, Any]] = None) -> RetrieverQueryEngine:
        llama_filters = None
        if filters:
            llama_filters = MetadataFilters(
                filters=[ExactMatchFilter(key=k, value=v) for k, v in filters.items()],
                condition=FilterOperator.AND
            )

        retriever = VectorIndexRetriever(
            index=self.index,
            similarity_top_k=similarity_top_k,
            filters=llama_filters
        )
        
        postprocessors = [SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)]
        
        query_engine = RetrieverQueryEngine(
            retriever=retriever,
            node_postprocessors=postprocessors
        )
        
        qa_prompt_template = PromptTemplate(
            """당신은 테스트케이스 전문가입니다. 주어진 테스트 스텝들을 바탕으로 사용자의 질문에 답변해주세요.

참고할 테스트 스텝들:
{context_str}

사용자 질문: {query_str}

답변은 한국어로 제공해주세요.

답변:"""
        )
        query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_template})
        return query_engine
    
    def test_lm_studio_direct(self, prompt: str) -> str:
        response = Settings.llm.complete(prompt)
        return response.text
    
    def print_search_results(self, results: List[Dict[str, Any]]):
        if not results:
            print("검색 결과가 없습니다.")
            return
        
        grouped = self.group_results_by_issue(results)
        
        for issue_key, steps in grouped.items():
            print(f"\n🔍 이슈: {issue_key}")
            print("=" * 50)
            
            for step in steps:
                step_idx = step["metadata"].get("step_index", "N/A")
                score = step.get("similarity_score", 0)
                print(f"\n📋 Step {step_idx} (유사도: {score:.3f})")
                print("-" * 30)
                print(step["content"])
    
    def print_qa_result(self, result: Dict[str, Any]):
        print(f"\n💡 LM Studio 답변:")
        print("=" * 50)
        print(result["answer"])
        
        if result["grouped_sources"]:
            print(f"\n📚 참고한 테스트케이스:")
            print("=" * 50)
            
            for issue_key, steps in result["grouped_sources"].items():
                print(f"\n🔗 {issue_key}")
                for step in steps:
                    step_idx = step["metadata"].get("step_index", "N/A")
                    score = step.get("score", 0)
                    print(f"  Step {step_idx} (점수: {score:.3f}): {step['content'][:100]}...")

    def save_json_to_file(self, filename: str, data: Dict[str, Any]):
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            print(f"✅ 모든 결과가 '{filename}' 파일에 JSON 형식으로 저장되었습니다.")
        except IOError as e:
            print(f"❌ JSON 파일 저장 중 오류 발생: {e}")

# 사용 예시
if __name__ == "__main__":
    rag = TestCaseRAGLlamaIndexLMStudio(
        lm_studio_url="http://127.0.0.1:1234/v1",
        lm_studio_model="qwen/qwen3-8b"
    )
    
    # ---------------------------------------------
    # LangChain 연동 예시
    # ---------------------------------------------
    print("\n--- LangChain 연동 테스트 ---")

    # LM Studio를 LangChain의 LLM으로 사용하기 위해 OpenAIWrapper 설정
    # LM Studio는 OpenAI API와 호환되므로 langchain_community.llms.OpenAI 사용
    lc_llm = OpenAI(
        base_url=rag.lm_studio_url,
        model_name=rag.lm_studio_model,
        temperature=rag.temperature,
        max_tokens=rag.max_tokens
    )
    print(f"✅ LangChain용 LM Studio LLM 설정 완료: {rag.lm_studio_url}")

    # LlamaIndex의 Retriever를 LangChain의 Retriever로 변환
    # LlamaIndex의 기본 Retriever 설정 (필터링 없이)
    llama_index_base_retriever = VectorIndexRetriever(
        index=rag.index,
        similarity_top_k=8,
        vector_store_query_mode="default"
    )
    # LlamaIndex Retriever를 LangChain Retriever로 래핑
    lc_retriever = LlamaIndexRetriever(llama_index_base_retriever)
    print("✅ LlamaIndex Retriever를 LangChain 호환 Retriever로 변환 완료")

    # LangChain RAG 프롬프트 템플릿 정의
    # (LlamaIndex 프롬프트와 비슷하지만 LangChain PromptTemplate 사용)
    lc_template = """당신은 테스트케이스 전문가입니다. 주어진 테스트 스텝들을 바탕으로 사용자의 질문에 정확하고 유용한 답변을 제공해주세요.

참고할 테스트 스텝들:
{context}

사용자 질문: {question}

답변 가이드라인:
1. 관련된 테스트 스텝들을 명확하게 설명해주세요
2. 각 테스트 스텝의 목적과 예상 결과를 포함해주세요
3. 테스트 데이터가 있다면 함께 제시해주세요
4. 이슈 키별로 그룹핑하여 전체적인 테스트 흐름을 보여주세요
5. 찾은 정보가 충분하지 않다면 추가로 필요한 정보를 제안해주세요
6. 답변은 한국어로 제공해주세요

답변:"""
    lc_prompt = LCPromptTemplate.from_template(lc_template)

    # LangChain RAG 체인 구축 (LCEL: LangChain Expression Language)
    # RAG 체인은 retriever로 문서를 찾고, 이 문서를 context로 사용하여 LLM이 답변을 생성하도록 합니다.
    lc_rag_chain = (
        {"context": lc_retriever, "question": RunnablePassthrough()}
        | lc_prompt
        | lc_llm
        | StrOutputParser()
    )
    print("✅ LangChain RAG 체인 구축 완료")

    # LangChain RAG 체인을 사용하여 질문하기
    langchain_question = "master admin 테스트의 주요 시나리오는 무엇인가요?"
    print(f"\n💬 LangChain RAG 체인으로 질문 처리 중: {langchain_question}")
    langchain_answer = lc_rag_chain.invoke(langchain_question)
    print(f"\n💡 LangChain RAG 답변:")
    print("=" * 50)
    print(langchain_answer)
    print("=" * 50)

    # ---------------------------------------------
    # 기존 테스트 코드 (JSON 저장을 위해 all_test_results에 추가)
    # ---------------------------------------------
    all_test_results = {}

    print("🤖 LM Studio 직접 테스트")
    direct_response = rag.test_lm_studio_direct("안녕하세요! 테스트 메시지입니다.")
    print(f"응답: {direct_response}")
    all_test_results["direct_lm_studio_test"] = {"query": "안녕하세요! 테스트 메시지입니다.", "response": direct_response}
    
    print("\n🔍 기본 검색 테스트")
    search_query_1 = "master admin 테스트 케이스"
    search_results_1 = rag.search_test_cases(search_query_1, top_k=5)
    rag.print_search_results(search_results_1)
    all_test_results["basic_search_test"] = {"query": search_query_1, "results": search_results_1}
    
    print("\n🔍 필터링 검색 테스트")
    search_query_2 = "테스트"
    filters_2 = {"issue_key": "COMMONR-380"}
    filtered_results_2 = rag.search_test_cases(search_query_2, filters=filters_2, top_k=5)
    rag.print_search_results(filtered_results_2)
    all_test_results["filtered_search_test"] = {"query": search_query_2, "filters": filters_2, "results": filtered_results_2}
    
    print("\n💬 LM Studio 질문 답변 테스트")
    qa_question_1 = "master admin 테스트는 어떻게 해야 하나요?"
    qa_result_1 = rag.ask_question(qa_question_1)
    rag.print_qa_result(qa_result_1)
    all_test_results["qa_test_1"] = {"question": qa_question_1, "response": qa_result_1}
    
    print("\n🔧 커스텀 쿼리 엔진 테스트")
    custom_engine_query = "master-slave master admin 테스트 방법"
    custom_engine_filters = {"issue_key": "COMMONR-380"} 
    custom_engine = rag.create_custom_query_engine(
        similarity_top_k=5,
        similarity_cutoff=0.8,
        filters=custom_engine_filters 
    )
    custom_response_obj = custom_engine.query(custom_engine_query)
    custom_response_text = str(custom_response_obj)
    print(f"커스텀 답변: {custom_response_text}")

    # custom_engine의 source_nodes도 저장하려면, ask_question처럼 내부 로직을 수정하여
    # source_nodes를 반환하도록 하거나, query_engine의 response 객체에서 직접 접근해야 합니다.
    # 여기서는 간단히 최종 텍스트만 저장합니다.
    all_test_results["custom_query_engine_test"] = {
        "query": custom_engine_query, 
        "filters": custom_engine_filters,
        "answer_text": custom_response_text,
    }
    
    print("\n💬 복잡한 질문 테스트")
    qa_question_2 = "master admin 테스트에서 주의해야 할 점과 필수 테스트 시나리오를 알려주세요."
    complex_qa_result = rag.ask_question(qa_question_2)
    rag.print_qa_result(complex_qa_result)
    all_test_results["complex_qa_test"] = {"question": qa_question_2, "response": complex_qa_result}
    
    print("\n📋 특정 이슈 전체 테스트 스텝 조회")
    issue_key_to_retrieve = "COMMONR-380"
    issue_steps = rag.get_test_case_by_issue(issue_key_to_retrieve)
    rag.print_search_results(issue_steps)
    all_test_results["get_all_steps_by_issue"] = {"issue_key": issue_key_to_retrieve, "steps": issue_steps}

    # LangChain 결과도 JSON에 추가
    all_test_results["langchain_rag_test"] = {
        "question": langchain_question,
        "answer": langchain_answer
    }

    output_filename = "rag_full_results.json"
    rag.save_json_to_file(output_filename, all_test_results)

In [ ]:
def main():
    """메인 실행 함수"""
    print("=== Jira to 임베딩 형식 변환 ===")
    print(f"Jira URL: {JIRA_URL}")
    print(f"사용자: {JIRA_USERNAME}")
    print(f"JQL 쿼리: {JQL_QUERY}")
    print()

    # 1. 이슈 키 목록 가져오기
    issue_keys = get_all_jira_issue_keys(JQL_QUERY)
    
    if not issue_keys:
        print("검색된 이슈가 없습니다.")
        return
    
    print(f"총 {len(issue_keys)}개의 이슈를 찾았습니다.")
    print()
    
    # 2. 모든 이슈 데이터를 임베딩 형태로 변환
    all_embedding_data = []
    successful_count = 0
    
    print("이슈 데이터를 가져오고 임베딩 형식으로 변환하는 중...")
    
    for i, issue_key in enumerate(issue_keys, 1):
        print(f"[{i}/{len(issue_keys)}] {issue_key} 처리 중...")
        
        # 임베딩용 형태로 변환
        embedding_objects = convert_to_embedding_format(issue_key)
        if embedding_objects:
            all_embedding_data.extend(embedding_objects)  # 리스트 확장
            # 개별 이슈별로 JSON 파일 저장
            json_to_save_file(embedding_objects, issue_key)
            successful_count += 1
        
        # API 호출 제한 고려
        time.sleep(0.1)
    
    print(f"\n성공적으로 처리된 이슈: {successful_count}/{len(issue_keys)}")
    
    # 3. 임베딩 데이터 출력 (처음 3개만 샘플로)
    print("\n=== 임베딩 데이터 샘플 ===")
    for i, item in enumerate(all_embedding_data[:3]):
        print(f"샘플 {i+1}:")
        print(f"ID: {item['id']}")
        print(f"Document: {item['document'][:100]}...")
        print(f"Metadata: {item['metadata']}")
        print("-" * 50)
    
    # 4. 통계
    print(f"\n=== 통계 ===")
    print(f"총 {len(all_embedding_data)}개의 임베딩 객체 생성")
    print(f"이슈당 평균 {len(all_embedding_data)/len(issue_keys):.1f}개의 테스트 스텝")
    
    # 평균 문서 길이
    if all_embedding_data:
        avg_doc_length = sum(len(item['document']) for item in all_embedding_data) / len(all_embedding_data)
        print(f"평균 문서 길이: {avg_doc_length:.0f} 문자")

if __name__ == "__main__":
    main()
    print("스크립트 실행 중...")

=== Jira to 임베딩 형식 변환 ===
Jira URL: https://jira.suprema.co.kr
사용자: dhwoo
JQL 쿼리: project = "COMMONR" AND issuetype = "Test"

JQL로 이슈 검색을 시작합니다: project = "COMMONR" AND issuetype = "Test"
  -> 현재까지 100 / 186 개 이슈 키를 가져왔습니다...
  -> 현재까지 186 / 186 개 이슈 키를 가져왔습니다...
총 186개의 이슈를 찾았습니다.

이슈 데이터를 가져오고 임베딩 형식으로 변환하는 중...
[1/186] COMMONR-380 처리 중...
 COMMONR-380 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[2/186] COMMONR-379 처리 중...
 COMMONR-379 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[3/186] COMMONR-378 처리 중...
 COMMONR-378 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[4/186] COMMONR-377 처리 중...
 COMMONR-377 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[5/186] COMMONR-376 처리 중...
 COMMONR-376 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[6/186] COMMONR-375 처리 중...
 COMMO